# Sentiment Analysis
> * Positive or Negative

In [1]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re

from collections import namedtuple
from gensim.models import doc2vec
import multiprocessing
from konlpy.tag import Mecab
from konlpy.utils import pprint
import numpy as np
from ckonlpy.tag import Twitter as ctwitter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
mecab = Mecab()
ct = ctwitter()

## Doc2Vec

In [2]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')

#### twitter

In [4]:
tagged_ct = pickle.load(open('./data/pre_data/pre_by_ct_data_tagged_run_docs.pickled','rb'))

In [5]:
train, test = train_test_split(tagged_ct, test_size=0.25, random_state=42)

In [ ]:
doc_vectorizer = doc2vec.Doc2Vec(size=2500, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(train)

for epoch in range(30):
    doc_vectorizer.train(train, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002  # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha  # fix the learning rate, no decay

#To save
doc_vectorizer.save('./model/doc2vec_size2500_epoch30_by_ct.model')
pprint(doc_vectorizer.most_similar('문재인/Noun'))
pprint(doc_vectorizer.most_similar('노무현/Noun'))
pprint(doc_vectorizer.most_similar('박근혜/Noun'))

In [6]:
model = doc2vec.Doc2Vec.load('./model/doc2vec_size2000_epoch30_by_ct.model')

In [ ]:
train_x = [model.infer_vector(doc.words) for doc in train]
train_y = [doc.tags[0] for doc in train]
test_x = [model.infer_vector(doc.words) for doc in test]
test_y = [doc.tags[0] for doc in test]

In [ ]:
classifier = LogisticRegression(random_state=1234)
classifier.fit(train_x, train_y)

In [ ]:
print( classifier.score(test_x, test_y) )

#### mecab

In [3]:
tagged_mecab = pickle.load(open('./data/pre_data/pre_by_mecab_data_tagged_run_docs.pickled','rb'))

In [4]:
train2, test2 = train_test_split(tagged_mecab, test_size=0.25, random_state=42)

In [ ]:
model2 = doc2vec.Doc2Vec.load('./model/doc2vec_size2000_epoch30_by_mecab.model')

In [ ]:
train2_x = [model2.infer_vector(doc.words) for doc in train2]
train2_y = [doc.tags[0] for doc in train2]
test2_x = [model2.infer_vector(doc.words) for doc in test2]
test2_y = [doc.tags[0] for doc in test2]

In [ ]:
classifier2 = LogisticRegression(random_state=1234)
classifier2.fit(train2_x, train2_y)

In [ ]:
print( classifier2.score(test2_x, test2_y) )